## Introduction

MNIST ("Modified National Institute of Standards and Technology") is the de facto “Hello World” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

In this competition, we aim to correctly identify digits from a dataset of tens of thousands of handwritten images. Kaggle has curated a set of tutorial-style kernels which cover everything from regression to neural networks. They hope to encourage us to experiment with different algorithms to learn first-hand what works well and how techniques compare.

## Approach

For this competition, we will be using Keras (with TensorFlow as our backend) as the main package to create a simple neural network to predict, as accurately as we can, digits from handwritten images. In particular, we will be calling the Functional Model API of Keras, and creating a 4-layered and 5-layered neural network.

Also, we will be experimenting with various optimizers: the plain vanilla Stochastic Gradient Descent optimizer and the Adam optimizer. However, there are many other parameters, such as training epochs which will we will not be experimenting with.

In addition, the choice of hidden layer units are completely arbitrary and may not be optimal. This is yet another parameter which we will not attempt to tinker with. Lastly, we introduce dropout, a form of regularisation, in our neural networks to prevent overfitting.

## Result

Following our simulations on the cross validation dataset, it appears that a 4-layered neural network, using 'Adam' as the optimizer along with a learning rate of 0.01, performs best. We proceed to introduce dropout in the model, and use the model to predict for the test set.

The test predictions (submitted to Kaggle) generated by our model predicts with an accuracy score of 97.600%, which places us at the top 55 percentile of the competition.

Importing key libraries, and reading data

In [1]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [3]:
df_train.head() # 784 features, 1 label

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Splitting into training and validation dataset

In [4]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.as_matrix().reshape(33600, 784) #(33600, 784)
X_cv = X_cv.as_matrix().reshape(8400, 784) #(8400, 784)

X_test = X_test.as_matrix().reshape(28000, 784)

## Data cleaning, normalization and selection

In [6]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


As the pixel intensities are currently between the range of 0 and 255, we proceed to normalize the features, using broadcasting. In addition, we proceed to convert our labels from a class vector to binary One Hot Encoded

In [7]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [8]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


## Model Fitting

We proceed by fitting several simple neural network models using Keras (with TensorFlow as our backend) and collect their accuracy. The model that performs the best on the validation set will be used as the model of choice for the competition.

Model 1: Simple Neural Network with 4 layers (300, 100, 100, 200)

In our first model, we will use the Keras library to train a neural network with the activation function set as ReLu. To determine which class to output, we will rely on the SoftMax function

In [9]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [10]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [11]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [13]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [14]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
4s - loss: 1.8541 - acc: 0.4983 - val_loss: 1.0046 - val_acc: 0.7602
Epoch 2/20
4s - loss: 0.6480 - acc: 0.8295 - val_loss: 0.4635 - val_acc: 0.8729
Epoch 3/20
4s - loss: 0.4092 - acc: 0.8834 - val_loss: 0.3616 - val_acc: 0.8980
Epoch 4/20
4s - loss: 0.3373 - acc: 0.9026 - val_loss: 0.3121 - val_acc: 0.9100
Epoch 5/20
4s - loss: 0.2979 - acc: 0.9139 - val_loss: 0.2893 - val_acc: 0.9169
Epoch 6/20
4s - loss: 0.2684 - acc: 0.9227 - val_loss: 0.2651 - val_acc: 0.9238
Epoch 7/20
4s - loss: 0.2453 - acc: 0.9296 - val_loss: 0.2557 - val_acc: 0.9257
Epoch 8/20
4s - loss: 0.2272 - acc: 0.9352 - val_loss: 0.2322 - val_acc: 0.9336
Epoch 9/20
3s - loss: 0.2101 - acc: 0.9378 - val_loss: 0.2175 - val_acc: 0.9363
Epoch 10/20
4s - loss: 0.1952 - acc: 0.9438 - val_loss: 0.2053 - val_acc: 0.9399
Epoch 11/20
4s - loss: 0.1828 - acc: 0.9468 - val_loss: 0.1954 - val_acc: 0.9427
Epoch 12/20
4s - loss: 0.1708 - acc: 0.9504 - val_loss: 0.1850 - val_

Using a 4 layer neural network with:

1. 20 training epochs
2. A training batch size of 100
3. Hidden layers set as (300, 100, 100, 200)
4. Learning rate of 0.1

Achieved a training score of around 96-98% and a test score of around 95 - 97%.

Can we do better if we were to change the optimizer? To find out, we use the Adam optimizer for our second model, while maintaining the same parameter values for all other parameters.

In [15]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
5s - loss: 0.3434 - acc: 0.8972 - val_loss: 0.1616 - val_acc: 0.9510
Epoch 2/20
5s - loss: 0.1260 - acc: 0.9610 - val_loss: 0.1195 - val_acc: 0.9645
Epoch 3/20
5s - loss: 0.0825 - acc: 0.9743 - val_loss: 0.1021 - val_acc: 0.9685
Epoch 4/20
5s - loss: 0.0603 - acc: 0.9807 - val_loss: 0.1212 - val_acc: 0.9639
Epoch 5/20
5s - loss: 0.0455 - acc: 0.9855 - val_loss: 0.1224 - val_acc: 0.9660
Epoch 6/20
5s - loss: 0.0394 - acc: 0.9864 - val_loss: 0.1034 - val_acc: 0.9700
Epoch 7/20
5s - loss: 0.0313 - acc: 0.9892 - val_loss: 0.1193 - val_acc: 0.9700
Epoch 8/20
5s - loss: 0.0275 - acc: 0.9914 - val_loss: 0.0979 - val_acc: 0.9735
Epoch 9/20
5s - loss: 0.0247 - acc: 0.9922 - val_loss: 0.0980 - val_acc: 0.9752
Epoch 10/20
5s - loss: 0.0192 - acc: 0.9938 - val_loss: 0.0926 - val_acc: 0.9769
Epoch 11/20
5s - loss: 0.0186 - acc: 0.9938 - val_loss: 0.1209 - val_acc: 0.9738
Epoch 12/20
5s - loss: 0.0240 - acc: 0.9920 - val_loss: 0.1114 - val_

As it turns out, it does appear to be the case that the optimizer plays a crucial part in the validation score. In particular, the model which relies on 'Adam' as its optimizer tend to perform 1.5 - 2.5% better on average. Going forward, we will use 'Adam' as our optimizer of choice.

What if we changed the learning rate from 0.1 to 0.01, or 0.5? Will it have any impact on the accuracy?
Model 2A

In [17]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
5s - loss: 0.3403 - acc: 0.8973 - val_loss: 0.1920 - val_acc: 0.9423
Epoch 2/20
5s - loss: 0.1242 - acc: 0.9618 - val_loss: 0.1203 - val_acc: 0.9645
Epoch 3/20
5s - loss: 0.0819 - acc: 0.9749 - val_loss: 0.1090 - val_acc: 0.9663
Epoch 4/20
5s - loss: 0.0612 - acc: 0.9802 - val_loss: 0.0993 - val_acc: 0.9704
Epoch 5/20
5s - loss: 0.0459 - acc: 0.9857 - val_loss: 0.1067 - val_acc: 0.9681
Epoch 6/20
5s - loss: 0.0352 - acc: 0.9880 - val_loss: 0.1074 - val_acc: 0.9705
Epoch 7/20
5s - loss: 0.0278 - acc: 0.9906 - val_loss: 0.0970 - val_acc: 0.9736
Epoch 8/20
5s - loss: 0.0242 - acc: 0.9920 - val_loss: 0.1112 - val_acc: 0.9736
Epoch 9/20
5s - loss: 0.0225 - acc: 0.9926 - val_loss: 0.1273 - val_acc: 0.9664
Epoch 10/20
5s - loss: 0.0245 - acc: 0.9923 - val_loss: 0.1239 - val_acc: 0.9706
Epoch 11/20
5s - loss: 0.0212 - acc: 0.9934 - val_loss: 0.1088 - val_acc: 0.9770
Epoch 12/20
5s - loss: 0.0180 - acc: 0.9943 - val_loss: 0.1090 - val_

Model 2B

In [19]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 5s - loss: 0.3354 - acc: 0.9026 - val_loss: 0.1438 - val_acc: 0.9570
Epoch 2/20
33600/33600 [==============================] - 5s - loss: 0.1231 - acc: 0.9619 - val_loss: 0.1276 - val_acc: 0.9612
Epoch 3/20
33600/33600 [==============================] - 5s - loss: 0.0809 - acc: 0.9749 - val_loss: 0.0969 - val_acc: 0.9719
Epoch 4/20
33600/33600 [==============================] - 5s - loss: 0.0561 - acc: 0.9819 - val_loss: 0.0867 - val_acc: 0.9744
Epoch 5/20
33600/33600 [==============================] - 5s - loss: 0.0475 - acc: 0.9849 - val_loss: 0.0882 - val_acc: 0.9743
Epoch 6/20
33600/33600 [==============================] - 5s - loss: 0.0363 - acc: 0.9887 - val_loss: 0.1015 - val_acc: 0.9727
Epoch 7/20
33600/33600 [==============================] - 5s - loss: 0.0288 - acc: 0.9908 - val_loss: 0.1124 - val_acc: 0.9713
Epoch 8/20
33600/33600 [==============================] - 5s -

The accuracy, as measured by the 3 different learning rates 0.01, 0.1 and 0.5 are around 98%, 97% and 98% respectively. As there are no considerable gains by changing the learning rates, we stick with the default learning rate of 0.01.

We proceed to fit a neural network with 5 hidden layers with the features in the hidden layer set as (300, 100, 100, 100, 200) respectively. To ensure that the two models are comparable, we will set the training epochs as 20, and the training batch size as 100.

In [21]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [22]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [23]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_5 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total para

In [24]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 5s - loss: 0.3611 - acc: 0.8924 - val_loss: 0.1889 - val_acc: 0.9433
Epoch 2/20
33600/33600 [==============================] - 5s - loss: 0.1267 - acc: 0.9604 - val_loss: 0.1305 - val_acc: 0.9625
Epoch 3/20
33600/33600 [==============================] - 5s - loss: 0.0850 - acc: 0.9738 - val_loss: 0.1177 - val_acc: 0.9660
Epoch 4/20
33600/33600 [==============================] - 5s - loss: 0.0598 - acc: 0.9815 - val_loss: 0.1045 - val_acc: 0.9713
Epoch 5/20
33600/33600 [==============================] - 5s - loss: 0.0463 - acc: 0.9855 - val_loss: 0.0926 - val_acc: 0.9757
Epoch 6/20
33600/33600 [==============================] - 5s - loss: 0.0369 - acc: 0.9884 - val_loss: 0.1009 - val_acc: 0.9732
Epoch 7/20
33600/33600 [==============================] - 5s - loss: 0.0339 - acc: 0.9895 - val_loss: 0.0982 - val_acc: 0.9760
Epoch 8/20
33600/33600 [==============================] - 5s -

Compared to our first model, adding an additional layer did not significantly improve the accuracy from our previous model. However, there are computational costs (in terms of complexity) in implementing an additional layer in our neural network. Given that the benefits of an additional layer are low while the costs are high, we will stick with the 4 layer neural network.

We now proceed to include dropout (dropout rate of 0.3) in our second model to prevent overfitting.

In [26]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [27]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [28]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

In [29]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 5s - loss: 0.5867 - acc: 0.8112 - val_loss: 0.1921 - val_acc: 0.9439
Epoch 2/20
33600/33600 [==============================] - 5s - loss: 0.2318 - acc: 0.9332 - val_loss: 0.1373 - val_acc: 0.9599
Epoch 3/20
33600/33600 [==============================] - 5s - loss: 0.1728 - acc: 0.9491 - val_loss: 0.1213 - val_acc: 0.9640
Epoch 4/20
33600/33600 [==============================] - 5s - loss: 0.1415 - acc: 0.9591 - val_loss: 0.1064 - val_acc: 0.9696
Epoch 5/20
33600/33600 [==============================] - 5s - loss: 0.1245 - acc: 0.9636 - val_loss: 0.1047 - val_acc: 0.9710
Epoch 6/20
33600/33600 [==============================] - 5s - loss: 0.1071 - acc: 0.9678 - val_loss: 0.0935 - val_acc: 0.9743
Epoch 7/20
33600/33600 [==============================] - 5s - loss: 0.1000 - acc: 0.9710 - val_loss: 0.0938 - val_acc: 0.9737
Epoch 8/20
33600/33600 [==============================] - 6s -

With a validation score of close to 98%, we proceed to use this model to predict for the test set.

In [31]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [32]:
test_pred.to_csv('mnist_submission.csv', index = False)

Using this model, we are able to achieve a score of 0.976, which places us at the top 55th percentile!